# Age and Gender prediction model using Vision Transformer

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
df = pd.read_pickle('withpixel_gender.pkl')

In [ ]:
pixels = np.array(df['pixels'].tolist())
ages = df['age'].values
genders = df['gender'].values

pixels = pixels.reshape(pixels.shape[0], 224, 224, 3)
pixels = pixels / 255.0

train_pixels, test_pixels, train_ages, test_ages, train_genders, test_genders = train_test_split(
    pixels, ages, genders, test_size=0.2, random_state=42)

In [ ]:
vit_url = 'https://tfhub.dev/google/vision_transformer/bit/1'
vit_layer = hub.KerasLayer(vit_url, trainable=False)

inputs = Input(shape=(224, 224, 3))
x = vit_layer(inputs)

# Age prediction head
age_head = Dense(101, activation='softmax', name='age_output')(x)

# Gender prediction head
gender_head = Dense(1, activation='sigmoid', name='gender_output')(x)

model = Model(inputs=inputs, outputs=[age_head, gender_head])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'age_output': 'sparse_categorical_crossentropy', 'gender_output': 'binary_crossentropy'},
              metrics={'age_output': 'accuracy', 'gender_output': 'accuracy'})

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_pixels,
                    {'age_output': train_ages, 'gender_output': train_genders},
                    validation_data=(test_pixels, {'age_output': test_ages, 'gender_output': test_genders}),
                    epochs=10,
                    batch_size=32)

In [ ]:
model.save('vit_age_gender_model.h5')